## This notebook contains all the scripts used to do segment level thermal sensitivity analysis and produce plots all plots required for presentation, report or article. The code block which are commented aren't necessary to do full e2e analysis, but are used to support report/article.

###  Import neccessary python libraries and pastis functions

In [ ]:
import os
os.chdir("/Users/asahoo/repos/PASTIS")
import time
from shutil import copy
from astropy.io import fits
import astropy.units as u
import hcipy
import numpy as np
import pastis.util as util    
from pastis.config import CONFIG_PASTIS 
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC 
from pastis.e2e_simulators.generic_segmented_telescopes import SegmentedAPLC
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import exoscene.image
import exoscene.star
import exoscene.planet
from exoscene.planet import Planet
from astropy.io import fits as pf
from matplotlib.colors import TwoSlopeNorm
import matplotlib.gridspec as gridspec
from pastis.temporal_analysis.close_loop_analysis import req_closedloop_calc_batch

### Read from config file

In [ ]:
nb_seg = CONFIG_PASTIS.getint('LUVOIR', 'nb_subapertures')
wvln = CONFIG_PASTIS.getfloat('LUVOIR', 'lambda') * 1e-9  # m #this doesn't matter, luvoir.wvln
diam = CONFIG_PASTIS.getfloat('LUVOIR', 'diameter')  # m
nm_aber = CONFIG_PASTIS.getfloat('LUVOIR', 'calibration_aberration') * 1e-9   # m
sampling = CONFIG_PASTIS.getfloat('LUVOIR', 'sampling')
coronagraph_design = CONFIG_PASTIS.get('LUVOIR','coronagraph_design')
optics_path_in_repo = CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo')
z_pup_downsample = CONFIG_PASTIS.getfloat('numerical', 'z_pup_downsample') 

### Define and create directory

In [ ]:
data_dir = "/Users/asahoo/Desktop/data_repos/harris_data"
repo_dir = "/Users/asahoo/repos/PASTIS"

overall_dir = util.create_data_path(data_dir, telescope='luvoir_'+coronagraph_design)
resDir = os.path.join(overall_dir, 'matrix_numerical')
print(resDir)

# Create necessary directories if they don't exist yet
os.makedirs(resDir, exist_ok=True)
os.makedirs(os.path.join(resDir, 'OTE_images'), exist_ok=True)
os.makedirs(os.path.join(resDir, 'psfs'), exist_ok=True)

### Load aperture files to make segmented mirror

In [ ]:
aper_path = CONFIG_PASTIS.get('LUVOIR','aperture_path_in_optics')
aper_ind_path = CONFIG_PASTIS.get('LUVOIR', 'indexed_aperture_path_in_optics')
aper_read = hcipy.read_fits(os.path.join(repo_dir,optics_path_in_repo,aper_path))
aper_ind_read = hcipy.read_fits(os.path.join(repo_dir,optics_path_in_repo,aper_ind_path))
pupil_grid = hcipy.make_pupil_grid(dims=aper_ind_read.shape[0], diameter=15)
aper = hcipy.Field(aper_read.ravel(), pupil_grid)
aper_ind = hcipy.Field(aper_ind_read.ravel(), pupil_grid)
wf_aper = hcipy.Wavefront(aper, wvln)

# Load segment positions from fits header
hdr = fits.getheader(os.path.join(repo_dir,optics_path_in_repo,aper_ind_path))

poslist = []
for i in range(nb_seg):
    segname = 'SEG' + str(i+1)
    xin = hdr[segname + '_X']
    yin = hdr[segname + '_Y']
    poslist.append((xin, yin))
    
poslist = np.transpose(np.array(poslist))
seg_pos = hcipy.CartesianGrid(hcipy.UnstructuredCoords(poslist))

### Plotting pupil functions, aperture function and index, pupil phase and amplitude $\color{red}{\text{(optional)}}$

In [ ]:
plt.figure(figsize=(20,10))

plt.subplot(2,3,1)
plt.title("pupil_grid")
plt.plot(pupil_grid.x, pupil_grid.y, '+')
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(2,3,2)
plt.title("aper")
hcipy.imshow_field(aper)
plt.tick_params(top=False, bottom=False, left=False, right=False,
                labelleft=False, labelbottom=False)
plt.colorbar()

plt.subplot(2,3,3)
plt.title("aper_ind")
hcipy.imshow_field(aper_ind)
plt.colorbar()

plt.subplot(2,3,4)
plt.title("wf_aper.phase")
hcipy.imshow_field(wf_aper.phase)
plt.colorbar()

plt.subplot(2,3,5)
plt.title("wf_aper.amplitude")
hcipy.imshow_field(wf_aper.amplitude)
plt.colorbar()

plt.subplot(2,3,6)
plt.title("seg_pos")
plt.plot(seg_pos.x, seg_pos.y, '+')
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()

### codeblock to plot aperture with segment numbers, (segment number is manually added using photoeditor applications) $\color{red}{\text{optional}}$

In [ ]:
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
pup_grid_plt = hcipy.make_pupil_grid(dims=1000, diameter=15)
luvoir_plt  = LuvoirA_APLC(optics_input, coronagraph_design, sampling)

plt.figure(figsize=(10, 10))
hcipy.imshow_field(luvoir_plt.aperture, cmap='bone')
plt.title('LUVOIR A segment numbering')
for i, par in enumerate(luvoir_plt.sm.seg_pos):
    pos = par * luvoir_plt.diam
    plt.annotate('text', xy=pos, xytext=pos-(0.2,0.1), color='black', fontweight='bold', size=12)


### Instantiate LUVOIR

In [ ]:
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
luvoir = LuvoirA_APLC(optics_input, coronagraph_design, sampling)

### load thermal modes files

In [ ]:
filepath = "/Users/asahoo/Documents/ultra/Sensitivities2.xlsx"
pad_orientation = np.pi/2*np.ones(nb_seg)

### create harris deformable mirror

In [ ]:
luvoir.create_segmented_harris_mirror(filepath,pad_orientation, thermal=True,mechanical=False,other=False) 
luvoir.harris_sm 

### create and plot single segment $\color{red}{\text{(optional)}}$

In [ ]:
segment = hcipy.hexagonal_aperture(luvoir.segment_circumscribed_diameter, np.pi/2) #function
segment_sampled = hcipy.evaluate_supersampled(segment,luvoir.pupil_grid, 10) #hcipy field

plt.figure(figsize=(5, 5))
hcipy.imshow_field(segment_sampled)
plt.colorbar()

### Create 'nb_seg' segments, this code may be useful if we want modulate the amplitude at the aperture $\color{red}{\text{(optional)}}$

In [ ]:
aper2, segs2 = hcipy.make_segmented_aperture(segment,luvoir.seg_pos, segment_transmissions=1, return_segments=True)
luvoir_segmented_pattern = hcipy.evaluate_supersampled(aper2, luvoir.pupil_grid, 10)
seg_evaluated = []
for seg_tmp in segs2:
    tmp_evaluated = hcipy.evaluate_supersampled(seg_tmp, luvoir.pupil_grid, 1)
    seg_evaluated.append(tmp_evaluated)

plt.figure(figsize=(15, 5))
plt.subplot(1,2,1)
plt.title("LUVOIR-A all segments")
hcipy.imshow_field(luvoir_segmented_pattern) 
plt.colorbar()

plt.subplot(1,2,2)
plt.title("LUVOIR-A single segment")
hcipy.imshow_field(seg_evaluated[75])
plt.colorbar()

### Plot Harris_Modal basis $\color{red}{\text{(optional)}}$

In [ ]:
harris_mode_basis = luvoir.create_segmented_harris_mirror(filepath,pad_orientation, thermal=True,
                                                          mechanical=False,other=False, return_mode=True)
from matplotlib.colors import TwoSlopeNorm
import matplotlib.gridspec as gridspec

plot_norm = TwoSlopeNorm(vcenter=0)

plt.figure(figsize=(40,20))
ax1 = plt.subplot2grid(shape=(2,6), loc=(0,0), colspan=2)
plt.title("Segment Level 1mK Faceplates Silvered", fontsize=30)
plot_norm1 = TwoSlopeNorm(vcenter=0, vmin=-10,  vmax =10)
plt.imshow(np.reshape(harris_mode_basis[0],(1000,1000))[163:263,804:904], cmap = 'RdBu', norm = plot_norm1)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

ax2 = plt.subplot2grid((2,6), (0,2), colspan=2)
plt.title("Segment Level 1mK bulk",fontsize=30)
plot_norm2 = TwoSlopeNorm(vcenter=0, vmin=-3,  vmax =1)
plt.imshow(np.reshape(harris_mode_basis[1],(1000,1000))[163:263,804:904],cmap = 'RdBu',norm = plot_norm2)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

ax3 = plt.subplot2grid((2,6), (0,4), colspan=2)
plt.title("Segment Level 1mK gradient radial",fontsize=30)
plot_norm3 = TwoSlopeNorm(vcenter=0, vmin=-1.5,  vmax =1.5) 
plt.imshow(np.reshape(harris_mode_basis[2],(1000,1000))[163:263,804:904],cmap = 'RdBu',norm = plot_norm3)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

ax4 = plt.subplot2grid((2,6), (1,1), colspan=2)
plt.title("Segment Level 1mK gradient X lateral",fontsize=30)
plot_norm4 = TwoSlopeNorm(vcenter=0, vmin=-2.5,  vmax =2.5) 
plt.imshow(np.reshape(harris_mode_basis[3],(1000,1000))[163:263,804:904],cmap = 'RdBu', norm = plot_norm4)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

ax5 = plt.subplot2grid((2,6), (1,3), colspan=2)
plt.title("Segment Level 1mK gradient Z axial",fontsize=30)
plot_norm5 = TwoSlopeNorm(vcenter=0,vmin=-3,  vmax =3)
plt.imshow(np.reshape(harris_mode_basis[4],(1000,1000))[163:263,804:904],cmap = 'RdBu', norm = plot_norm5)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

plt.tight_layout()
plt.savefig(os.path.join(resDir, 'harris_modal_basis.png'))

### Flatten the harris DM 

In [ ]:
luvoir.harris_sm.flatten()

### Calculate the unaberrated coro and direct PSFs in INTENSITY

In [ ]:
unaberrated_coro_psf, ref = luvoir.calc_psf(ref=True, display_intermediate=False, norm_one_photon=True)
norm = np.max(ref)
dh_intensity = (unaberrated_coro_psf / norm) * luvoir.dh_mask
contrast_floor = np.mean(dh_intensity[np.where(luvoir.dh_mask != 0)])
print(f'contrast floor: {contrast_floor}')

### Plot the direct and coronagraphic PSF in units of $\lambda /D$ $\color{red}{\text{(optional)}}$

In [ ]:
ideal_coro_psf = np.reshape(unaberrated_coro_psf, [115,115])
ideal_psf = np.reshape(ref, [115,115])
ind_max = np.unravel_index(np.argmax(np.abs(ideal_psf), axis=None), np.abs(ideal_psf).shape)

#pf.writeto((os.path.join(resDir, 'ideal_psf.fits')), np.abs(ideal_psf))

for i in range(0,115,20):
    print(i, (i-ind_max[0])*(1/5)) #from the ideal_psf image, 10pixels = 2l/d, thus 1 pixel = (1/5)l/d
   
    
x_pixels = [0,17,37,57,77,97,114]
x_ld = [r"-11.4$\lambda/D$",r"-8.0$\lambda/D$", r"-4.0$\lambda/D$", 0, r"4.0$\lambda/D$", r"8.0$\lambda/D$",r"11.4$\lambda/D$"]

plt.figure(figsize=(22,9))
plt.subplot(1,2,1)
plt.title("Coronagraphic PSF", fontsize=25)
plt.imshow(np.log10(np.abs(ideal_coro_psf)), cmap='magma')
plt.xticks(x_pixels, x_ld, fontsize =13)
plt.yticks(x_pixels, x_ld, fontsize =13)
cbar = plt.colorbar(ticks = np.linspace(-16,-2,7,endpoint=False))
cbar.ax.set_yticklabels([r'$10^{-16}$', r'$10^{-14}$', r'$10^{-12}$', r'$10^{-10}$',
                         r'$10^{-8}$', r'$10^{-6}$', r'$10^{-4}$'], fontsize=15)
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',direction='in')
plt.xlabel("in units of "r"$\lambda /D$")
plt.ylabel("in units of "r"$\lambda /D$")


plt.subplot(1,2,2)
plt.title("PSF without Coronagraph", fontsize=25)
plt.imshow(np.log10(np.abs(ideal_psf)), cmap ='magma')
plt.xticks(x_pixels, x_ld, fontsize=13)
plt.yticks(x_pixels, x_ld, fontsize=13)
cbar = plt.colorbar(ticks = np.linspace(-12,0,6,endpoint=False))
cbar.ax.set_yticklabels([r'$10^{-12}$', r'$10^{-10}$', r'$10^{-8}$', r'$10^{-6}$', 
                         r'$10^{-4}$', r'$10^{-2}$'], fontsize=15)
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',direction='out')
plt.xlabel("in units of "r"$\lambda /D$")
plt.ylabel("in units of "r"$\lambda /D$")

plt.tight_layout()
plt.savefig(os.path.join(resDir, 'ideal_coro_psf.png'))


### Plotting Optical train $\color{red}{\text{(optional)}}$

In [ ]:
lyot_stop_train = np.reshape(luvoir.lyotstop, [1000,1000])
fpm_optical_train = np.reshape(luvoir.fpm, [150,150])

fpm_train = np.ones((1000,1000))
for i in range(425,576):
    for j in range(425,576):
        fpm_train[i,j] = fpm_optical_train[(425-i), (425-j)]


plt.figure(figsize=(25,17))
ax1 = plt.subplot2grid(shape=(2,6), loc=(0,0), colspan=2)
plt.title("LUVOIR-A Aperture", fontsize=45)
hcipy.imshow_field(wf_aper.amplitude)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)

ax2 = plt.subplot2grid((2,6), (0,2), colspan=2)
plt.title("Narrow Angle Apodizer", fontsize=45)
hcipy.imshow_field(luvoir.apodizer)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)

ax3 = plt.subplot2grid((2,6), (0,4), colspan=2)
plt.title("Focal Plane Mask",fontsize=45)
plt.imshow(fpm_train)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)

ax4 = plt.subplot2grid((2,6), (1,1), colspan=2)
plt.title("Lyot Stop",fontsize=45)
hcipy.imshow_field(luvoir.lyotstop)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)

ax5 = plt.subplot2grid((2,6), (1,3), colspan=2)
plt.title("Unabberated Coronagraphic PSF",fontsize=45)
plt.imshow(np.log(np.abs(ideal_coro_psf)))
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)

plt.tight_layout()
plt.savefig(os.path.join(resDir, 'optical_train.png'))

### Calculate the unaberrated coro and direct PSFs in E-FIELDS

In [ ]:
nonaberrated_coro_psf, ref, efield = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
Efield_ref = nonaberrated_coro_psf.electric_field

### Poking each segment with a thermal aberration and generating the subsequent electric field

In [ ]:
print('Generating the E-fields for harris modes in science plane')
print(f'Calibration aberration used: {nm_aber} m')

start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []
n_harris = luvoir.harris_sm.num_actuators
for pp in range(0, n_harris):
    print(f'Working on mode {pp}/{n_harris}')
    
    # Apply calibration aberration to used mode
    harris_mode = np.zeros(n_harris)
    harris_mode[pp] = (nm_aber)/2 
    luvoir.harris_sm.actuators  = harris_mode
    
    # Calculate coronagraphic E-field and add to lists
    aberrated_coro_psf, inter = luvoir.calc_psf(display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

### Construct the PASTIS matrix from the E-fields

In [ ]:
mat_fast = np.zeros([n_harris, n_harris])   # create empty matrix

for i in range(0, n_harris):
    for j in range(0, n_harris):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref ) * np.conj(focus_fieldS[j].electric_field -Efield_ref ))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_fast[i, j] = contrast

### Plotting pastis matrix with scientific notation colorbar

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
clist = [(0.1, 0.6, 1.0), (0.05, 0.05, 0.05), (0.8, 0.5, 0.1)]
blue_orange_divergent = LinearSegmentedColormap.from_list("custom_blue_orange", clist)

plt.figure(figsize=(10,8))
norm_mat = TwoSlopeNorm(vcenter=0,vmin=-0.005*1e-7,  vmax = 0.005*1e-7)
plt.imshow(mat_fast, cmap=blue_orange_divergent, norm = norm_mat)
plt.title(r"PASTIS matrix $M$", fontsize=20)
plt.xlabel("Mode Index",fontsize=20)
plt.ylabel("Mode Index",fontsize=20)
plt.tick_params(labelsize=15)
cbar = plt.colorbar(fraction=0.046, pad=0.04)
cbar.set_label(r"in units of $1/{nm^2}$",fontsize =15)
plt.tight_layout()

plt.savefig(os.path.join(resDir, 'pastis_matrix.png'))

### Save PASTIS matrix, Real and Imaginary part of electric field at the focus as a fits file

### Save PASTIS matrix as fits  

In [ ]:
filename_matrix = 'PASTISmatrix_n_harris_' + str(n_harris)
hcipy.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

filename_matrix = 'EFIELD_Re_matrix_n_harris_' + str(n_harris)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))


filename_matrix = 'EFIELD_Im_matrix_n_harris_' + str(n_harris)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

end_time = time.time()
print('Runtime for harris modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

### Generating eigenvalues and eigenvectors for the PASTIS matrix

In [ ]:
evals, evecs = np.linalg.eig(matrix_pastis)
sorted_evals = np.sort(evals)
sorted_indices = np.argsort(evals)
sorted_evecs = evecs[:, sorted_indices]

### codeblock to plot eigenvalues $\lambda_{p}$

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(sorted_evals)
plt.semilogy()
plt.xlabel('Mode Index',fontsize=20)
plt.ylabel('Eigenvalues (in units of contrast/'r'wave$^2$)',fontsize=20)
plt.tick_params(top=True, bottom=True, left=True, right=True,
                labelleft=True, labelbottom=True, labelsize=20)
plt.tick_params(axis='both',direction='in')
plt.tick_params(axis='both',which='major',length=10, width=2)

plt.savefig(os.path.join(resDir, 'eigenvalues.png'))

### Codeblock to plot sorted eigenmodes ${b}_{p}$

In [ ]:
emodes = []
eunit = 1e-9
for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    
    harris_coeffs = eunit*sorted_evecs[:, mode]/2
    luvoir.harris_sm.actuators = harris_coeffs
    wf_harris_sm = luvoir.harris_sm(luvoir.wf_aper)
    emodes.append(wf_harris_sm.phase)
    
#to plot a single eigen mode use following 
hcipy.imshow_field(emodes[2])

### Set the desired target_contrast for which you want to calculate tolerances 

In [ ]:
c_target_log = -11
c_target = 10**(c_target_log)

### Generating and saving tolerance/ $\mu$ values for each segement, each mode

In [ ]:
mu_map_harris = np.sqrt(((c_target) / (n_harris)) / (np.diag(matrix_pastis)))
np.savetxt(os.path.join(resDir, 'mu_map_harris_%s.csv'%c_target), mu_map_harris, delimiter=',')

#mu_map_harris = np.genfromtxt('os.path.join(resDir, 'mu_map_harris.csv')',delimiter=',')

### Codeblock to plot tolerances per mode per segments

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Modal constraints to achieve a dark hole contrast of "r"$10^{%d}$"%c_target_log, fontsize=30)
plt.ylabel("Weight per mode (in units of nm)",fontsize =20)
plt.xlabel("Mode index", fontsize=20)
plt.tick_params(top=True, bottom=True, left=True, right=True,labelleft=True, labelbottom=True, labelsize=20)
plt.plot(mu_map_harris)
plt.tight_layout()
plt.savefig(os.path.join(resDir, 'mu_map_harris_%s.png'%c_target))

### plot max mode contribution(s) from the static-contrast target and eigen values

In [ ]:
sigma = np.sqrt(((c_target)) / (600 * sorted_evals))
plt.figure(figsize=(15,8))
plt.title("Max mode contribution(s) from the static-contrast target %s and eigen values"%c_target, fontsize=20)
plt.ylabel(r"$\tilde{b}_{p}$ (in units of nm)",fontsize =20)
plt.xlabel("Mode index", fontsize=20)
plt.tick_params(top=True, bottom=True, left=True, right=True,labelleft=True, labelbottom=True, labelsize=20)
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',direction='in')
ax1.tick_params(axis='x',direction='out')
plt.ylim(-5,40)
plt.plot(sigma)
plt.savefig(os.path.join(resDir, 'sigma_%s.png'%c_target))

### Codeblock to generate cumulative contrast

In [ ]:
cont_cum_pastis = []
for maxmode in range(sorted_evecs.shape[0]):
    
    aber = np.nansum(sorted_evecs[:, :maxmode+1] * sigma[:maxmode+1], axis=1)
    aber *= u.nm
    contrast_matrix = util.pastis_contrast(aber, matrix_pastis) + contrast_floor
    cont_cum_pastis.append(contrast_matrix)

### codeblock to plot cumulative contrast

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(cont_cum_pastis)
plt.xlabel("Mode Index", fontsize =20)
plt.ylabel("Cumulative contrast", fontsize =20)
plt.tick_params(top=True, bottom=True, left=True, right=True,labelleft=True, labelbottom=True, labelsize=20)
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',direction='in')
plt.tight_layout()

plt.savefig(os.path.join(resDir, 'cont_cum_pastis_%s.png'% c_target))

### codeblock to plot list of individual contrast 

In [ ]:
cont_ind_pastis = []
for maxmode in range(sorted_evecs.shape[0]):
    aber = sorted_evecs[:, maxmode] * sigma[maxmode]
    aber *=u.nm
    contrast_matrix = util.pastis_contrast(aber, matrix_pastis)
    cont_ind_pastis.append(contrast_matrix)

In [ ]:
plt.figure(figsize=(20,10))

plt.plot((cont_ind_pastis))
plt.semilogy()
plt.xlabel("Mode Index",fontsize=20)
plt.ylabel("List of Individual contrast",fontsize=20)
plt.tick_params(top=True, bottom=True, left=True, right=True,labelleft=True, labelbottom=True, labelsize=20)
plt.yscale('log')

plt.savefig(os.path.join(resDir, 'cont_ind_pastis_%s.png'%c_target))

### Create tolerance maps (later to be converted in pm or mk scales )

In [ ]:
N_harris = 5 #number of harris modes, (thermal only)
harris_coeffs_numaps = np.zeros([N_harris,n_harris])
harris_modes_std = mu_map_harris

for qq in range(N_harris):
    harris_coeffs_tmp = np.zeros([n_harris])
    for kk in range(nb_seg):
        harris_coeffs_tmp[qq+(kk)*N_harris] = harris_modes_std[qq+(kk)*N_harris]
    harris_coeffs_numaps[qq] = harris_coeffs_tmp #arranged 600 elements into 5*120 elements 
    
harris_coeffs_table = np.zeros([N_harris, nb_seg])
for qq in range(N_harris):
    for kk in range(nb_seg):
        harris_coeffs_table[qq,kk] = mu_map_harris[qq+(kk)*N_harris] #numpy ndarray 120 #nm

nu_maps = []
for qq in range(N_harris):
    harris_coeffs = harris_coeffs_numaps[qq]
    luvoir.harris_sm.actuators = harris_coeffs*nm_aber/ 2 # m
    nu_maps.append(luvoir.harris_sm.surface) #field  

### in picometer 

In [ ]:
plt.figure(figsize=(40,20))
ax1 = plt.subplot2grid(shape=(2,6), loc=(0,0), colspan=2)
plt.title("Segment Level 1mK Faceplates Silvered", fontsize=30)
plot_norm1 = TwoSlopeNorm(vcenter=0,vmin=-5,  vmax =5) 
hcipy.imshow_field((nu_maps[0])*1e12, cmap = 'RdBu', norm = plot_norm1)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

ax2 = plt.subplot2grid((2,6), (0,2), colspan=2)
plt.title("Segment Level 1mK bulk",fontsize=30)
plot_norm2 = TwoSlopeNorm(vcenter=0, vmin=-10,  vmax =10)
hcipy.imshow_field((nu_maps[1])*1e12,cmap = 'RdBu',norm = plot_norm2)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

ax3 = plt.subplot2grid((2,6), (0,4), colspan=2)
plt.title("Segment Level 1mK gradiant radial",fontsize=30)
plot_norm3 = TwoSlopeNorm(vcenter=0, vmin=-10,  vmax =10) 
hcipy.imshow_field((nu_maps[2])*1e12,cmap = 'RdBu',norm = plot_norm3)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

ax4 = plt.subplot2grid((2,6), (1,1), colspan=2)
plt.title("Segment Level 1mK gradient X lateral",fontsize=30)
plot_norm4 = TwoSlopeNorm(vcenter=0, vmin=-10,  vmax =10) 
hcipy.imshow_field((nu_maps[3])*1e12,cmap = 'RdBu', norm = plot_norm4)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

ax5 = plt.subplot2grid((2,6), (1,3), colspan=2)
plt.title("Segment Level 1mK gradient Z axial",fontsize=30)
plot_norm5 = TwoSlopeNorm(vcenter=0,vmin=-10,  vmax =10)
hcipy.imshow_field((nu_maps[4])*1e12,cmap = 'RdBu', norm = plot_norm5)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("pm",fontsize =30)

plt.tight_layout()
plt.savefig(os.path.join(resDir, 'stat_mu_maps_pm_%s.png'%c_target))

### in mK 

In [ ]:
luvoir2 = LuvoirA_APLC(optics_input, coronagraph_design, sampling)
luvoir2.create_segmented_mirror(1)
luvoir2.sm.actuators =harris_coeffs_table[0]

luvoir3 = LuvoirA_APLC(optics_input, coronagraph_design, sampling)
luvoir3.create_segmented_mirror(1)
luvoir3.sm.actuators = harris_coeffs_table[1]

luvoir4 = LuvoirA_APLC(optics_input, coronagraph_design, sampling)
luvoir4.create_segmented_mirror(1)
luvoir4.sm.actuators = harris_coeffs_table[2]

luvoir5 = LuvoirA_APLC(optics_input, coronagraph_design, sampling)
luvoir5.create_segmented_mirror(1)
luvoir5.sm.actuators = harris_coeffs_table[3]

luvoir6 = LuvoirA_APLC(optics_input, coronagraph_design, sampling)
luvoir6.create_segmented_mirror(1)
luvoir6.sm.actuators = harris_coeffs_table[4]

In [ ]:
print("Allowable temperature change (in mK)","\n")
print("Faceplate silvered: ",np.sqrt(np.mean(np.square(luvoir2.sm.actuators)))*1000)
print("Bulk: ", np.sqrt(np.mean(np.square(luvoir3.sm.actuators)))*1000)
print("Gradient Radial:",np.sqrt(np.mean(np.square(luvoir4.sm.actuators)))*1000)
print("Gradient X Lateral: ",np.sqrt(np.mean(np.square(luvoir5.sm.actuators)))*1000)
print("Gradient Z axial: ",np.sqrt(np.mean(np.square(luvoir6.sm.actuators)))*1000)


In [ ]:
from matplotlib.colors import TwoSlopeNorm
import matplotlib.gridspec as gridspec


plt.figure(figsize=(40,20))
ax1 = plt.subplot2grid(shape=(2,6), loc=(0,0), colspan=2)
plt.title("Faceplates Silvered", fontsize=30)
hcipy.imshow_field((luvoir2.sm.surface)*1000, cmap = 'RdBu',vmin=0,  vmax =15)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("mK",fontsize =30)


ax2 = plt.subplot2grid((2,6), (0,2), colspan=2)
plt.title("Bulk",fontsize=30)
hcipy.imshow_field((luvoir3.sm.surface)*1000, cmap = 'RdBu', vmin=0,vmax =70)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("mK",fontsize =30)



ax3 = plt.subplot2grid((2,6), (0,4), colspan=2)
plt.title("Gradiant Radial",fontsize=30)
hcipy.imshow_field((luvoir4.sm.surface)*1000,cmap = 'RdBu',vmin=0,  vmax =140)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("mK",fontsize =30)



ax4 = plt.subplot2grid((2,6), (1,1), colspan=2)
plt.title("Gradient X lateral",fontsize=30)
hcipy.imshow_field((luvoir5.sm.surface)*1000,cmap = 'RdBu', vmin=0,  vmax =30)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("mK",fontsize =30)



ax5 = plt.subplot2grid((2,6), (1,3), colspan=2)
plt.title("Gradient Z axial",fontsize=30)
hcipy.imshow_field((luvoir6.sm.surface)*1000,cmap = 'RdBu', vmin=0,  vmax =50)
plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=30)
cbar.set_label("mK",fontsize =30)

plt.tight_layout()
plt.savefig(os.path.join(resDir, 'mu_map_mK_%s.png'%c_target)

# Temporal Ananlysis

### Define stellar parameters

In [ ]:
npup = int(np.sqrt(luvoir.pupil_grid.x.shape[0]))
nimg = int(np.sqrt(luvoir.focal_det.x.shape[0]))

# Getting the flux together
sptype = 'A0V' # Put this on config
Vmag = 5.0 # Put this in loop
minlam = 500 * u.nanometer # Put this on config
maxlam = 600 * u.nanometer # Put this on config
dark_current = 0   #0.000072 #es per s  #put this on config
CIC = 0            #0.00076 #electrons per sec #put this on config
star_flux = exoscene.star.bpgs_spectype_to_photonrate(spectype=sptype, Vmag=Vmag, minlam=minlam.value, maxlam=maxlam.value)
Nph = star_flux.value*15**2*np.sum(luvoir.apodizer**2) / npup**2


In [ ]:
harris_mode = np.zeros(n_harris)
luvoir.harris_sm.actuators  = harris_mode
nonaberrated_coro_psf, refshit,inter_ref = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
Efield_ref = nonaberrated_coro_psf.electric_field

### out-of-band reference electric field seen by out-of-band wavefront sensor

In [ ]:
N_pup_z = int(luvoir.pupil_grid.shape[0] / z_pup_downsample) #N_pup_z = 100,used to define out-of-band efield
grid_zernike = hcipy.field.make_pupil_grid(N_pup_z, diameter=luvoir.diam)

In [ ]:
# plt.figure(figsize=(10,10))
# plt.title("grid_zernike") #hcipy cartesian grid, later on used to define Efield_ref_OBWFS
# plt.plot(grid_zernike.x, grid_zernike.y, '+')
# plt.xlabel('x')
# plt.ylabel('y')

In [ ]:
harris_mode = np.zeros(n_harris)
luvoir.harris_sm.actuators  = harris_mode
harris_ref2 = luvoir.calc_out_of_band_wfs(norm_one_photon=True)
harris_ref2_sub_real = hcipy.field.subsample_field(harris_ref2.real, z_pup_downsample, grid_zernike, statistic='mean')
harris_ref2_sub_imag = hcipy.field.subsample_field(harris_ref2.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref_OBWFS = (harris_ref2_sub_real + 1j*harris_ref2_sub_imag) * z_pup_downsample

In [ ]:
# plt.figure(figsize = (20,10))
# plt.subplot(1,2,1)
# hcipy.imshow_field(Efield_ref_OBWFS.real, cmap ='RdBu')
# plt.colorbar()
# plt.subplot(1,2,2)
# hcipy.imshow_field(Efield_ref_OBWFS.imag, cmap ='RdBu')
# plt.colorbar()

### calculate nyquist grid

In [ ]:
nyquist_sampling = 2.

# Actual grid for LUVOIR images
grid_test = hcipy.make_focal_grid(
            luvoir.sampling,
            luvoir.imlamD,
            pupil_diameter=luvoir.diam,
            focal_length=1,
            reference_wavelength=luvoir.wvln,
        )

# Actual grid for LUVOIR images that are nyquist sampled
grid_det_subsample = hcipy.make_focal_grid(
            nyquist_sampling,
            np.floor(luvoir.imlamD),
            pupil_diameter=luvoir.diam,
            focal_length=1,
            reference_wavelength=luvoir.wvln,
        )
n_nyquist = np.int(np.sqrt(grid_det_subsample.x.shape[0]))

### Setting up nyquist sampled dark hole mask

In [ ]:
### Dark hole mask
design = 'small'

dh_outer_nyquist = hcipy.circular_aperture(2 * luvoir.apod_dict[design]['owa'] * luvoir.lam_over_d)(grid_det_subsample)
dh_inner_nyquist = hcipy.circular_aperture(2 * luvoir.apod_dict[design]['iwa'] * luvoir.lam_over_d)(grid_det_subsample)
dh_mask_nyquist = (dh_outer_nyquist - dh_inner_nyquist).astype('bool')

dh_size = len(np.where(luvoir.dh_mask != 0)[0])
dh_size_nyquist = len(np.where(dh_mask_nyquist != 0)[0])
dh_index = np.where(luvoir.dh_mask != 0)[0]
dh_index_nyquist = np.where(dh_mask_nyquist != 0)[0]

### electric field as seen by out-of-band zernike wavefront sensor

In [ ]:
E0_OBWFS = np.zeros([N_pup_z*N_pup_z,1,2])
E0_OBWFS[:,0,0] = Efield_ref_OBWFS.real
E0_OBWFS[:,0,1] = Efield_ref_OBWFS.imag

### coronagraphic electric field in the focal plane

In [ ]:
E0_coron = np.zeros([nimg*nimg,1,2])
E0_coron[:,0,0] = Efield_ref.real
E0_coron[:,0,1] = Efield_ref.imag
# E0_coron_nyquist = np.zeros([n_nyquist*n_nyquist,1,2])
# tmp0 = hcipy.interpolation.make_linear_interpolator_separated(Efield_ref, grid=grid_test)
# Efield_ref_nyquist = (luvoir.sampling/nyquist_sampling)**2*tmp0(grid_det_subsample)
# E0_coron_nyquist[:,0,0] = Efield_ref_nyquist.real
# E0_coron_nyquist[:,0,1] = Efield_ref_nyquist.imag
# E0_coron_DH = np.zeros([dh_size,1,2])
# E0_coron_DH[:,0,0] = Efield_ref.real[dh_index]
# E0_coron_DH[:,0,1] = Efield_ref.imag[dh_index]
# E0_coron_DH_nyquist = np.zeros([dh_size_nyquist,1,2])
# E0_coron_DH_nyquist[:,0,0] = Efield_ref_nyquist.real[dh_index_nyquist]
# E0_coron_DH_nyquist[:,0,1] = Efield_ref_nyquist.real[dh_index_nyquist]

### Defining Sensitvity matrix after coronagraph plane/ image plane??

In [ ]:
filename_matrix = 'EFIELD_Re_matrix_n_harris_' + str(n_harris) + '.fits'
G_harris_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix = 'EFIELD_Im_matrix_n_harris_' + str(n_harris) + '.fits'
G_harris_imag = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))

# G_coron_harris_nyquist= np.zeros([n_nyquist*n_nyquist,2,n_harris])
# for pp in range(0, n_harris):
#     tmp0 = G_harris_real[pp] + 1j*G_harris_imag[pp]
#     tmp1 = hcipy.interpolation.make_linear_interpolator_separated(tmp0, grid=grid_test)
#     tmp2 = (luvoir.sampling/nyquist_sampling)**2*tmp1(grid_det_subsample)
#     G_coron_harris_nyquist[:,0,pp] = tmp2.real - Efield_ref_nyquist.real
#     G_coron_harris_nyquist[:,1,pp] = tmp2.real - Efield_ref_nyquist.imag

In [ ]:
# G_coron_harris_DH= np.zeros([dh_size,2,n_harris])
# for pp in range(0, n_harris):
#     G_coron_harris_DH[:,0,pp] = G_harris_real[pp,dh_index] - Efield_ref.real[dh_index]
#     G_coron_harris_DH[:,1,pp] = G_harris_imag[pp,dh_index] - Efield_ref.imag[dh_index]

In [ ]:
# G_coron_harris_DH_nyquist= np.zeros([dh_size_nyquist,2,n_harris])
# for pp in range(0, n_harris):
#     tmp0 = G_harris_real[pp] + 1j*G_harris_imag[pp]
#     tmp1 = hcipy.interpolation.make_linear_interpolator_separated(tmp0, grid=grid_test)
#     tmp2 = (luvoir.sampling/nyquist_sampling)**2*tmp1(grid_det_subsample)
#     G_coron_harris_DH_nyquist[:,0,pp-1] = tmp2.real[dh_index_nyquist] - Efield_ref_nyquist.real[dh_index_nyquist]
#     G_coron_harris_DH_nyquist[:,1,pp-1] = tmp2.real[dh_index_nyquist] - Efield_ref_nyquist.imag[dh_index_nyquist]

In [ ]:
G_coron_harris= np.zeros([nimg*nimg,2,n_harris])
for pp in range(0, n_harris):
    G_coron_harris[:,0,pp] = G_harris_real[pp] - Efield_ref.real
    G_coron_harris[:,1,pp] = G_harris_imag[pp] - Efield_ref.imag

### Calculate out-of-band sensitivity matrix or G_OBWFS

In [ ]:
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

for pp in range(0, n_harris):
    print(pp)
    harris_modes = np.zeros(n_harris)
    harris_modes[pp] = (nm_aber) / 2
    luvoir.harris_sm.actuators  = harris_modes
    harris_meas = luvoir.calc_out_of_band_wfs(norm_one_photon=True)
    harris_meas_sub_real = hcipy.field.subsample_field(harris_meas.real, z_pup_downsample, grid_zernike, statistic='mean')
    harris_meas_sub_imag = hcipy.field.subsample_field(harris_meas.imag, z_pup_downsample, grid_zernike, statistic='mean')
    focus_field1 = harris_meas_sub_real + 1j * harris_meas_sub_imag
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
filename_matrix = 'EFIELD_OBWFS_Re_matrix_num_harris_' + str(n_harris)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

filename_matrix = 'EFIELD_OBWFS_Im_matrix_num_harris_' + str(n_harris)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_OBWFS_Re_matrix_num_harris_' + str(n_harris)+'.fits'
G_OBWFS_real = fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))
filename_matrix = 'EFIELD_OBWFS_Im_matrix_num_harris_' + str(n_harris)+'.fits'
G_OBWFS_imag =  fits.getdata(os.path.join(overall_dir, 'matrix_numerical', filename_matrix))

In [ ]:
G_OBWFS= np.zeros([N_pup_z*N_pup_z,2,n_harris])
for pp in range(0, n_harris):
    G_OBWFS[:,0,pp] = G_OBWFS_real[pp]*z_pup_downsample - Efield_ref_OBWFS.real
    G_OBWFS[:,1,pp] = G_OBWFS_imag[pp]*z_pup_downsample - Efield_ref_OBWFS.imag

In [ ]:
def req_closedloop_calc_recursive(Gcoro, Gsensor, E0coro, E0sensor, Dcoro, Dsensor, t_exp, flux, Q, Niter, dh_mask,
                                  norm):
    P = np.zeros(Q.shape)  # WFE modes covariance estimate
    r = Gsensor.shape[2]
    N = Gsensor.shape[0]
    N_img = Gcoro.shape[0]
    c = 1
    # Iterations of ALGORITHM 1
    contrast_hist = np.zeros(Niter)
    intensity_WFS_hist = np.zeros(Niter)
    cal_I_hist = np.zeros(Niter)
    eps_hist = np.zeros([Niter, r])
    averaged_hist = np.zeros(Niter)
    contrasts = []
    for pp in range(Niter):
        eps = np.random.multivariate_normal(np.zeros(r), P + Q * t_exp).reshape((1, 1, r))  # random modes
        G_eps = np.sum(Gsensor * eps, axis=2).reshape((N, 1, 2 * c)) + E0sensor  # electric field
        G_eps_squared = np.sum(G_eps * G_eps, axis=2, keepdims=True)
        G_eps_G = np.matmul(G_eps, Gsensor)
        G_eps_G_scaled = G_eps_G / np.sqrt(G_eps_squared + Dsensor / flux / t_exp)  # trick to save RAM
        cal_I = 4 * flux * t_exp * np.einsum("ijk,ijl->kl", G_eps_G_scaled, G_eps_G_scaled)  # information matrix
        P = np.linalg.inv(np.linalg.inv(P + Q * t_exp / 2) + cal_I)
        #         P = np.linalg.inv(cal_I)

        # Coronagraph
        G_eps_coron = np.sum(Gcoro * eps, axis=2).reshape((N_img, 1, 2 * c)) + E0coro
        G_eps_coron_squared = np.sum(G_eps_coron * G_eps_coron, axis=2, keepdims=True)
        intensity = G_eps_coron_squared * flux * t_exp + Dcoro

        # Wavefront sensor
        intensity_WFS = G_eps_squared * flux * t_exp + Dsensor

        # Archive
        test_DH0 = intensity[:, 0, 0] * luvoir.dh_mask
        test_DH = np.mean(test_DH0[np.where(test_DH0 != 0)])
        contrasts.append(test_DH / flux / t_exp / norm)
        intensity_WFS_hist[pp] = np.sum(intensity_WFS) / flux
        cal_I_hist[pp] = np.mean(cal_I) / flux
        eps_hist[pp] = eps
        averaged_hist[pp] = np.mean(contrasts)
        #         print("est. contrast", np.mean(contrasts))

        outputs = {'intensity_WFS_hist': intensity_WFS_hist,
                   'cal_I_hist': cal_I_hist,
                   'eps_hist': eps_hist,
                   'averaged_hist': averaged_hist,
                   'contrasts': contrasts}
    return outputs

In [ ]:
def req_closedloop_calc_batch(Gcoro, Gsensor, E0coro, E0sensor, Dcoro, Dsensor, t_exp, flux, Q, Niter, dh_mask, norm):
    P = np.zeros(Q.shape)  # WFE modes covariance estimate
    r = Gsensor.shape[2]
    N = Gsensor.shape[0]
    N_img = Gcoro.shape[0]
    c = 1
    # Iterations of ALGORITHM 1
    contrast_hist = np.zeros(Niter)
    intensity_WFS_hist = np.zeros(Niter)
    cal_I_hist = np.zeros(Niter)
    eps_hist = np.zeros([Niter, r])
    averaged_hist = np.zeros(Niter)
    contrasts = []
    for pp in range(Niter):
        eps = np.random.multivariate_normal(np.zeros(r), P + Q * t_exp).reshape((1, 1, r))  # random modes
        G_eps = np.sum(Gsensor * eps, axis=2).reshape((N, 1, 2 * c)) + E0sensor  # electric field
        G_eps_squared = np.sum(G_eps * G_eps, axis=2, keepdims=True)
        G_eps_G = np.matmul(G_eps, Gsensor)
        G_eps_G_scaled = G_eps_G / np.sqrt(G_eps_squared + Dsensor / flux / t_exp)  # trick to save RAM
        cal_I = 4 * flux * t_exp * np.einsum("ijk,ijl->kl", G_eps_G_scaled, G_eps_G_scaled)  # information matrix
        #         P = np.linalg.inv(np.linalg.inv(P+Q*t_exp/2) + cal_I)
        P = np.linalg.pinv(cal_I)

        # Coronagraph
        G_eps_coron = np.sum(Gcoro * eps, axis=2).reshape((N_img, 1, 2 * c)) + E0coro
        G_eps_coron_squared = np.sum(G_eps_coron * G_eps_coron, axis=2, keepdims=True)
        intensity = G_eps_coron_squared * flux * t_exp + Dcoro

        # Wavefront sensor
        intensity_WFS = G_eps_squared * flux * t_exp + Dsensor

        # Archive
        test_DH0 = intensity[:, 0, 0] * luvoir.dh_mask
        test_DH = np.mean(test_DH0[np.where(test_DH0 != 0)])
        contrasts.append(test_DH / flux / t_exp / norm)
        intensity_WFS_hist[pp] = np.sum(intensity_WFS) / flux
        cal_I_hist[pp] = np.mean(cal_I) / flux
        eps_hist[pp] = eps
        averaged_hist[pp] = np.mean(contrasts)
    #         print("est. contrast", np.mean(contrasts))
    #         print("est. contrast", np.mean(contrasts))

    outputs = {'intensity_WFS_hist': intensity_WFS_hist,
               'cal_I_hist': cal_I_hist,
               'eps_hist': eps_hist,
               'averaged_hist': averaged_hist,
               'contrasts': contrasts}

    return outputs


### Starting close loop plots

In [ ]:
# Running a bunch of tests for time series
flux = Nph
Qharris = np.diag(np.asarray(mu_map_harris**2))

Ntimes = 20
TimeMinus = -2
TimePlus = 5.5 #3.5
Nwavescale = 8
WaveScaleMinus = -2
WaveScalePlus = 1
Nflux = 3
fluxPlus = 10
fluxMinus = 0

timeVec = np.logspace(TimeMinus,TimePlus,Ntimes)
WaveVec = np.logspace(WaveScaleMinus,WaveScalePlus,Nwavescale)
fluxVec = np.linspace(fluxMinus,fluxPlus,Nflux)
wavescaleVec = np.logspace(WaveScaleMinus,WaveScalePlus,Nwavescale)

In [ ]:
#10**(-fluxVec/2.5)

### fixed scaling factor, iterating over variable stellar magnitude

In [ ]:
# result_mv =[]

# #i=-1
# for StarMag in range (2,7,1):
#     #i=i+1
#     print('Harris modes with batch OBWFS and noise %f'% StarMag, "i",i) 
#     wavescale = 1.
#     niter = 10
#     timer1 = time.time()
#     #StarMag = 5.0
#     #j=-1
#     for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
#         j=j+1
#         Starfactor = 10**(-StarMag/2.5)
#         print(tscale)
#         tmp0 = req_closedloop_calc_batch(G_coron_harris, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
#                                                  dark_current+CIC/tscale, tscale, flux*Starfactor, 0.1*wavescale**2*Qharris,
#                                                  niter, luvoir.dh_mask, norm)    
#         tmp1 = tmp0['averaged_hist']
#         n_tmp1 = len(tmp1)
#         result_mv.append(tmp1[n_tmp1-1])


In [ ]:
# texp = np.logspace(TimeMinus, TimePlus, Ntimes)

# plt.figure(figsize =(15,10))
# plt.plot(texp,result_mv[0:20]-contrast_floor, label=r'$m_{v}=2$')
# plt.plot(texp,result_mv[20:40]-contrast_floor, label=r'$m_{v}=3$')
# plt.plot(texp,result_mv[40:60]-contrast_floor, label=r'$m_{v}=4$')
# plt.plot(texp,result_mv[60:80]-contrast_floor, label=r'$m_{v}=5$')
# plt.plot(texp,result_mv[80:100]-contrast_floor, label=r'$m_{v}=6$')
# plt.xlabel("$t_{WFS}$ in secs",fontsize=20)
# plt.ylabel("$\Delta$ contrast",fontsize=20)
# #plt.ylabel("Raw Contrast",fontsize=20)
# plt.yscale('log')
# plt.xscale('log')
# plt.legend(loc = 'lower right',fontsize=20)
# plt.tick_params(top=True, bottom=True, left=True, 
#                 right=True,labelleft=True, labelbottom=True,
#                 labelsize=20)
# plt.tick_params(axis='both',which='major',length=10, width=2)
# plt.tick_params(axis='both',which='minor',length=6, width=2)
# plt.grid()
# plt.savefig('/Users/asahoo/Documents/ultra/temp_plots/cont_mv_delta.png')
# plt.show()
#plt.savefig('/Users/asahoo/Documents/ultra/temp_plots/cont_mv.png')



# plt.figure(figsize =(15,10))
# plt.plot(texp,result_mv[0:20], label=r'$m_{v}=2$')
# plt.plot(texp,result_mv[20:40], label=r'$m_{v}=3$')
# plt.plot(texp,result_mv[40:60], label=r'$m_{v}=4$')
# plt.plot(texp,result_mv[60:80], label=r'$m_{v}=5$')
# plt.plot(texp,result_mv[80:100], label=r'$m_{v}=6$')
# plt.xlabel("$t_{WFS}$ in secs")
# plt.ylabel("$\Delta$ contrast")
# plt.yscale('log')
# plt.xscale('log')
# plt.legend(loc = 'lower right')

# plt.show()

### fixed stellar magitude, iterating only the scaling factor

In [ ]:
res = np.zeros([Ntimes, Nwavescale, Nflux, 1])
result_wf_test =[]

#i=-1
for wavescale in range (1,15,2):
    #i=i+1
    print('Harris modes with batch OBWFS and noise %f'% wavescale, "i",i)  
    niter = 10
    timer1 = time.time()
    StarMag = 0.0
    #j=-1
    for tscale in np.logspace(TimeMinus, TimePlus, Ntimes):
        j=j+1
        Starfactor = 10**(-StarMag/2.5)
        print(tscale)
        tmp0 = req_closedloop_calc_batch(G_coron_harris, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
                                                 dark_current+CIC/tscale, tscale, flux*Starfactor, 0.0001*wavescale**2*Qharris,
                                                 niter, luvoir.dh_mask, norm)    
        tmp1 = tmp0['averaged_hist']
        n_tmp1 = len(tmp1)
        result_wf_test.append(tmp1[n_tmp1-1])


In [ ]:
### 0.000001*wavescale**2 is in nm**2

In [ ]:
delta_wf = []
for wavescale in range (1,15,2):
    wf = 1e3*np.sqrt(0.0001*wavescale**2)
    delta_wf.append(wf)
    
texp = np.logspace(TimeMinus, TimePlus, Ntimes)
font = {'family': 'serif','color' : 'black','weight': 'normal','size'  :  20}

plt.figure(figsize =(15,10))
plt.title('Target contrast = %s, Vmag= %s'%(c_target, Vmag),fontdict=font)
plt.plot(texp,result_wf_test[0:20]-contrast_floor, label=r'$\Delta_{wf}= %d\ pm$'%(delta_wf[0]))
plt.plot(texp,result_wf_test[20:40]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[1]))
plt.plot(texp,result_wf_test[40:60]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[2]))
plt.plot(texp,result_wf_test[60:80]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[3]))
plt.plot(texp,result_wf_test[80:100]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[4]))
plt.plot(texp,result_wf_test[100:120]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[5]))
plt.plot(texp,result_wf_test[120:140]-contrast_floor, label=r'$\Delta_{wf}=%d\ pm$'%(delta_wf[6]))
plt.xlabel("$t_{WFS}$ in secs",fontsize=20)
plt.ylabel("$\Delta$ contrast",fontsize=20)
plt.yscale('log')
plt.xscale('log')
plt.legend(loc = 'upper center',fontsize=20)
plt.tick_params(top=True, bottom=True, left=True, 
                right=True,labelleft=True, labelbottom=True,
                labelsize=20)
plt.tick_params(axis='both',which='major',length=10, width=2)
plt.tick_params(axis='both',which='minor',length=6, width=2)
plt.grid()
#plt.savefig('/Users/asahoo/Documents/ultra/temp_plots/cont_wf_03_01.png')
plt.show()

### random uncorrelated noise and extra tolerancing analysis

In [ ]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# luvoir_test = LuvoirA_APLC(optics_input, coronagraph_design, sampling)
# luvoir_test.create_segmented_harris_mirror(filepath,pad_orientation,thermal =True,mechanical=False,other=False) 
# luvoir_test.harris_sm

In [ ]:
# from astropy.io import fits as pf
# import matplotlib.ticker as ticker

# del_contrast = []
# aberrated_contrast = []
# rms_wfs = []
# for i in range(1, 1000):    
#     harris_mode = np.zeros(n_harris)
#     harris_mode = np.random.normal(0, mu_map_harris*nm_aber, n_harris)
#     rms = np.sqrt(np.mean(harris_mode**2))*1e12
#     luvoir_test.harris_sm.actuators  = harris_mode/2
#     aberrated_coro_psf_t, inter_t = luvoir_test.calc_psf(display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
#     focal_int = np.zeros((115,115))
#     focal_int = np.array(np.reshape(aberrated_coro_psf_t.amplitude,(115,115)))
#     focal_cont = (np.square(focal_int))/norm
#     dh_intensity = ((np.square(aberrated_coro_psf_t.amplitude))/ norm) * luvoir_test.dh_mask
#     contrast_floor_aber = np.mean(dh_intensity[np.where(luvoir_test.dh_mask != 0)])
#     delta_contrast = contrast_floor_aber-contrast_floor

#     del_contrast.append(delta_contrast)
#     aberrated_contrast.append(contrast_floor_aber)
#     rms_wfs.append(rms)
    
#     print(rms, contrast_floor_aber, delta_contrast)

#     textstr1 = '\n'.join(("RMS (in pm) = %.2f" % (np.sqrt(np.mean(harris_mode**2))*1e12),))
#     textstr2 = r'${\langle c\rangle}_{DH} = $'+ str("{:.2e}".format(contrast_floor_aber))
#     textstr3 = r'$\Delta_{{\langle c\rangle}_{DH}} = $'+ str("{:.2e}".format(delta_contrast))

#     plt.figure(figsize=(24,10))            
#     ax1=plt.subplot(1,2,1)
#     plt.title("Wavefront", fontsize =20)
#     props = dict(boxstyle='round', facecolor='white', alpha=1.0)
#     props2 = dict(boxstyle='round', facecolor='red', alpha=1.0)
#     ax1.text(0.01, 0.99, textstr1, transform=ax1.transAxes, fontsize=15,verticalalignment='top', bbox=props)
#     ax1.text(1.15, 1.01, "pm", transform=ax1.transAxes, fontsize=15,verticalalignment='top')
#     plt_norm = TwoSlopeNorm(vcenter=0, vmin=-0.3, vmax=0.3)
#     hcipy.imshow_field((inter_t['harris_seg_mirror']).phase*1000, mask=luvoir_test.aperture, cmap='RdBu', norm=plt_norm)
#     plt.tick_params(top=False, bottom=True, left=True, right=False ,labelleft=True, labelbottom=True, labelsize=15)
#     plt.tick_params(axis='both',which='major',length=10, width=2)
#     plt.tick_params(axis='both',which='minor',length=6, width=2)
#     cbar = plt.colorbar()
#     cbar.ax.tick_params(labelsize=15)
#     cbar.set_label("pm",fontsize =20)


#     ax2 = plt.subplot(1,2,2)
#     plt.title("Coronagraphic PSF", fontsize =20)
#     props = dict(boxstyle='round', facecolor='white', alpha=1.0)
#     ax2.text(0.01, 0.99, textstr2, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
#     ax2.text(0.01, 0.93, textstr3, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
#     plt.imshow(np.log10(focal_cont),cmap='magma')
#     cbar = plt.colorbar(ticks = np.linspace(-14,-2,6,endpoint=False))
#     cbar.ax.set_yticklabels([r'$10^{-14}$', r'$10^{-12}$', r'$10^{-10}$', 
#                              r'$10^{-8}$', r'$10^{-6}$',r'$10^{-4}$'], fontsize=15)
#     plt.tick_params(top=False, bottom=True, left=True, right=False ,labelleft=True, labelbottom=True, labelsize=15)
#     plt.tick_params(axis='both',which='major',length=10, width=2)
#     plt.tick_params(axis='both',which='minor',length=6, width=2)

#     #plt.show()
#     plt.savefig('/Users/asahoo/Documents/ultra/march_31st/gif_all/plot_%d.png'%i)
#     plt.close()
    #pf.writeto('/Users/asahoo/Desktop/data_repos/harris_data/ball_del_06/pupil_%d.fits'%pp,pupil_phase)
    


In [ ]:
# import scipy

# #mean = np.sqrt(np.mean((harris_coeffs_numaps[4]*nm_aber)**2))*1e12*np.sqrt(5)
# mean = np.sqrt(np.mean((mu_map_harris*nm_aber)**2))*1e12
# print(mean)

# plt.figure(figsize =(40,15))
# plt.subplot(1,3,1)
# _, bins, _ = plt.hist(rms_wfs, 100, density=1, alpha=1)
# mu, sigma = scipy.stats.norm.fit(rms_wfs)
# best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
# plt.plot(bins, best_fit_line, lw='4',label = r'$\mu$ =%.2f, $\sigma$= %.2f'%(mu,sigma))
# #plt.axvline(mean, c='r', ls='-.', lw='3')
# plt.xlabel("RMS Wavefront Error (in pm)", fontsize =20)
# plt.ylabel("Frequency",fontsize =15)
# plt.tick_params(axis='both', which='both', length=6, width=2, labelsize=20)
# plt.legend(fontsize=25)
# #plt.xlim(0,3)

# plt.subplot(1,3,2)
# _, bins, _ = plt.hist(np.log10(aberrated_contrast), 100, density=1, alpha=1)
# mu, sigma = scipy.stats.norm.fit(np.log10(aberrated_contrast))
# best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
# plt.plot(bins, best_fit_line, lw='4',label = r'$\mu$ =%.2f, $\sigma$= %.2f'%(mu,sigma))
# #plt.axvline(np.log10(contrast_floor), c='r', ls='-.', lw='3')
# plt.xlabel("Mean contrast in DH",fontsize =15)
# plt.ylabel("Frequency",fontsize =15)
# plt.tick_params(axis='both', which='both', length=6, width=2, labelsize=20)
# plt.legend(fontsize=25)
# #plt.xlim(-10.37,-10.34)

# plt.subplot(1,3,3)
# _, bins, _ = plt.hist(np.log10(del_contrast), 100, density=1, alpha=1)
# mu, sigma = scipy.stats.norm.fit(np.log10(del_contrast))
# best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
# plt.plot(bins, best_fit_line, lw='4',label = r'$\mu$ =%.2f, $\sigma$= %.2f'%(mu,sigma))
# #plt.axvline(np.log10(c_target), c='r', ls='-.', lw='3')
# plt.xlabel("Change in contrast in DH",fontsize =20)
# plt.tick_params(axis='both', which='both', length=6, width=2, labelsize=20)
# plt.ylabel("Frequency",fontsize =15)
# plt.legend(fontsize=25)
# plt.savefig('/Users/asahoo/Documents/ultra/march_31st/hist_mu_map_new.png')

In [ ]:
# 1*10**(-11.70), 10**(-11.10), np.sqrt(5)*11.805254684082996

### Code block to fill rhs of the table 

In [ ]:
# Qharris = np.diag(np.asarray(mu_map_harris**2))

In [ ]:
# 19.4/1e3

In [ ]:
# Qharris0 = np.diag(np.asarray(harris_coeffs_numaps[0]**2))
# Qharris1 = np.diag(np.asarray(harris_coeffs_numaps[1]**2))
# Qharris2 = np.diag(np.asarray(harris_coeffs_numaps[2]**2))
# Qharris3 = np.diag(np.asarray(harris_coeffs_numaps[3]**2))
# Qharris4 = np.diag(np.asarray(harris_coeffs_numaps[4]**2))

In [ ]:
# tscale = 10
# StarMag = 0.0
# Starfactor = 10**(-StarMag/2.5)
# niter = 10
# #wavescale = 130
# c_contrast0 =  req_closedloop_calc_batch(G_coron_harris, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
#                                                  dark_current+CIC/tscale, tscale, flux*Starfactor,  0.0225*Qharris0,
#                                                  niter, luvoir.dh_mask, norm)
# result_c0 = []
# c0 = c_contrast0 ['averaged_hist']
# n_tmp1 = len(c0)
# result_c0.append(c0[n_tmp1-1])
# print(result_c0-contrast_floor)

In [ ]:
# tscale = 1
# StarMag = 0.0
# Starfactor = 10**(-StarMag/2.5)
# niter = 10
#wavescale = 130
# c_contrast0 =  req_closedloop_calc_batch(G_coron_harris, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
#                                                  dark_current+CIC/tscale, tscale, flux*Starfactor,  0.711*Qharris0,
#                                                  niter, luvoir.dh_mask, norm)
# c_contrast1 =  req_closedloop_calc_batch(G_coron_harris, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
#                                                  dark_current+CIC/tscale, tscale, flux*Starfactor,  0.711*Qharris1,
#                                                  niter, luvoir.dh_mask, norm)
# c_contrast2 =  req_closedloop_calc_batch(G_coron_harris, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
#                                                  dark_current+CIC/tscale, tscale, flux*Starfactor,  0.711*Qharris2,
#                                                  niter, luvoir.dh_mask, norm)
# c_contrast3 =  req_closedloop_calc_batch(G_coron_harris, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
#                                                  dark_current+CIC/tscale, tscale, flux*Starfactor,  0.711*Qharris3,
#                                                  niter, luvoir.dh_mask, norm)
# c_contrast4 =  req_closedloop_calc_batch(G_coron_harris, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
#                                                  dark_current+CIC/tscale, tscale, flux*Starfactor,  0.711*Qharris4,
#                                                  niter, luvoir.dh_mask, norm)
# c_contrast =  req_closedloop_calc_batch(G_coron_harris, G_OBWFS, E0_coron, E0_OBWFS, dark_current+CIC/tscale,
#                                                  dark_current+CIC/tscale, tscale, flux*Starfactor, 0.711*Qharris,
#                                                  niter, luvoir.dh_mask, norm)

In [ ]:
# result_c0 = []
# c0 = c_contrast0 ['averaged_hist']
# n_tmp1 = len(c0)
# result_c0.append(c0[n_tmp1-1])
# print(result_c0-contrast_floor)

# result_c1 = []
# c1 = c_contrast1 ['averaged_hist']
# n_tmp1 = len(c1)
# result_c1.append(c1[n_tmp1-1])
# print(result_c1-contrast_floor)

# result_c2 = []
# c2 = c_contrast2 ['averaged_hist']
# n_tmp1 = len(c2)
# result_c2.append(c0[n_tmp1-1])
# print(result_c2-contrast_floor)

# result_c3 = []
# c3 = c_contrast3 ['averaged_hist']
# n_tmp1 = len(c3)
# result_c3.append(c3[n_tmp1-1])
# print(result_c3-contrast_floor)

# result_c4 = []
# c4 = c_contrast4 ['averaged_hist']
# n_tmp1 = len(c4)
# result_c4.append(c4[n_tmp1-1])
# print(result_c4-contrast_floor)

# result_c = []
# c_total = c_contrast['averaged_hist']
# n_tmp1 = len(c_total)
# result_c.append(c_total[n_tmp1-1])
# print(result_c-contrast_floor)


## In mk/s

In [ ]:
# delta_wf = 1 #unit less
# t_wfs = 1 #in sec

# plt.figure(figsize=(40,20))
# ax1 = plt.subplot2grid(shape=(2,6), loc=(0,0), colspan=2)
# plt.title("Faceplates Silvered", fontsize=30)
# hcipy.imshow_field((luvoir2.sm.surface)*1000*delta_wf*(1/t_wfs), cmap = 'RdBu')
# plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
# cbar = plt.colorbar()
# cbar.ax.tick_params(labelsize=30)
# cbar.set_label("mK/s",fontsize =30)


# ax2 = plt.subplot2grid((2,6), (0,2), colspan=2)
# plt.title("Bulk",fontsize=30)
# hcipy.imshow_field((luvoir3.sm.surface)*1000*delta_wf*(1/t_wfs), cmap = 'RdBu')
# plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
# cbar = plt.colorbar()
# cbar.ax.tick_params(labelsize=30)
# cbar.set_label("mK/s",fontsize =30)



# ax3 = plt.subplot2grid((2,6), (0,4), colspan=2)
# plt.title("Gradiant Radial",fontsize=30)
# hcipy.imshow_field((luvoir4.sm.surface)*1000*delta_wf*(1/t_wfs),cmap = 'RdBu')
# plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
# cbar = plt.colorbar()
# cbar.ax.tick_params(labelsize=30)
# cbar.set_label("mK/s",fontsize =30)



# ax4 = plt.subplot2grid((2,6), (1,1), colspan=2)
# plt.title("Gradient X lateral",fontsize=30)
# hcipy.imshow_field((luvoir5.sm.surface)*1000*delta_wf*(1/t_wfs),cmap = 'RdBu')
# plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
# cbar = plt.colorbar()
# cbar.ax.tick_params(labelsize=30)
# cbar.set_label("mK/s",fontsize =30)



# ax5 = plt.subplot2grid((2,6), (1,3), colspan=2)
# plt.title("Gradient Z axial",fontsize=30)
# hcipy.imshow_field((luvoir6.sm.surface)*1000*delta_wf*(1/t_wfs),cmap = 'RdBu')
# plt.tick_params(top=False, bottom=False, left=False, right=False,labelleft=False, labelbottom=False)
# cbar = plt.colorbar()
# cbar.ax.tick_params(labelsize=30)
# cbar.set_label("mK/s",fontsize =30)

# plt.tight_layout()
# plt.savefig('/Users/asahoo/Documents/ultra/report_11_01_21/mu_map_mKs.png')

In [ ]:
# delta_wf = 0.13 #nm
# t_wfs = 25 #in sec

# print(np.min(five_vec_1)*1000*delta_wf*(1/t_wfs), 
#       np.max(five_vec_1)*1000*delta_wf*(1/t_wfs), np.mean(five_vec_1)*1000*delta_wf*(1/t_wfs)
#       , np.std(five_vec_1)*1000*delta_wf*(1/t_wfs), '\n')
# print(np.min(five_vec_2)*1000*delta_wf*(1/t_wfs), 
#       np.max(five_vec_2)*1000*delta_wf*(1/t_wfs), np.mean(five_vec_2)*1000*delta_wf*(1/t_wfs)
#       , np.std(five_vec_2)*1000*delta_wf*(1/t_wfs), '\n')
# print(np.min(five_vec_3)*1000*delta_wf*(1/t_wfs), 
#       np.max(five_vec_3)*1000*delta_wf*(1/t_wfs), np.mean(five_vec_3)*1000*delta_wf*(1/t_wfs)
#       , np.std(five_vec_3)*1000*delta_wf*(1/t_wfs), '\n')
# print(np.min(five_vec_4)*1000*delta_wf*(1/t_wfs), 
#       np.max(five_vec_4)*1000*delta_wf*(1/t_wfs), np.mean(five_vec_4)*1000*delta_wf*(1/t_wfs)
#       , np.std(five_vec_4)*1000*delta_wf*(1/t_wfs), '\n')
# print(np.min(five_vec_5)*1000*delta_wf*(1/t_wfs), 
#       np.max(five_vec_5)*1000*delta_wf*(1/t_wfs), np.mean(five_vec_5)*1000*delta_wf*(1/t_wfs)
#       , np.std(five_vec_5)*1000*delta_wf*(1/t_wfs), '\n')

### Plotting intensity after poking each segment with harris mode

In [ ]:
# luvoir_test = LuvoirA_APLC(optics_input, coronagraph_design, sampling)
# luvoir_test.create_segmented_harris_mirror(filepath,pad_orientation, thermal = True,mechanical=False,other=False) 
# luvoir_test.harris_sm

#harris_mode = np.zeros(n_harris)
#harris_mode[116] = nm_aber
#luvoir_test.harris_sm.actuators  = harris_mode
#hcipy.imshow_field(((10*luvoir_test.harris_sm.surface+5*1e-8*luvoir_segmented_pattern)))
#plt.colorbar()

In [ ]:
# from astropy.io import fits as pf
# import matplotlib.ticker as ticker

# del_contrast = []
# contrast_floor_aber = []

# for pp in range(0, 1):
#     #print(f'Working on mode {5*pp+4}',{pp})
    
#     # Apply calibration aberration to used mode
#     harris_mode = np.zeros(n_harris)
#     harris_mode[5*pp+4] = five_vec_5[pp]*1e-9 #m
#     luvoir_test.harris_sm.actuators  = harris_mode/2

#     #Calculate coronagraphic E-field and add to lists
#     aberrated_coro_psf_t, inter_t = luvoir_test.calc_psf(display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
#     pupil_phase = np.zeros((1000,1000))
#     pupil_phase = np.array(np.reshape(inter_t['harris_seg_mirror'].phase,(1000,1000)))
#     focal_int = np.zeros((115,115))
#     focal_int = np.array(np.reshape(aberrated_coro_psf_t.amplitude,(115,115)))
#     focal_cont = (np.square(focal_int))/norm
#     #focal_int2 = np.square(focal_int) - (np.square(np.array(np.reshape(unaberrated_coro_psf,(115,115))))/norm)
    
#     dh_intensity2 = (((unaberrated_coro_psf))/norm)*luvoir_test.dh_mask
#     contrast_floor_aber2 = np.mean(dh_intensity2[np.where(luvoir_test.dh_mask != 0)])
#     dh_intensity = ((np.square(aberrated_coro_psf_t.amplitude))/ norm) * luvoir_test.dh_mask
#     contrast_floor_aber = np.mean(dh_intensity[np.where(luvoir_test.dh_mask != 0)])
#     delta_contrast = contrast_floor_aber-contrast_floor_aber2
#     print(contrast_floor_aber2, contrast_floor_aber, delta_contrast)
    
#     change = np.square(focal_cont) - (np.square(np.array(np.reshape(unaberrated_coro_psf,(115,115))))/norm)
    
    
#     image_diff = (np.square(aberrated_coro_psf_t.amplitude) - unaberrated_coro_psf)/norm
#     change_mean = np.mean(image_diff[np.where(luvoir_test.dh_mask != 0)])
#     print(change_mean)
    #del_contrast.append(delta)
    #contrast_floor_aber.append(contrast_floor_aber)
    
#     plt.figure(figsize=(14,5))
#     plt.subplot(1,3,1)
#     hcipy.imshow_field((inter_t['harris_seg_mirror']).phase, mask=luvoir_test.aperture, cmap='RdBu')
#     plt.title("Wavefront")
#     plt.colorbar()
    #cbar = plt.colorbar()
    #cbar.ax.tick_params(labelsize = 15)
    #cbar.set_label("radians",fontsize =15)


#     plt.subplot(1,3,2)
#     plt.imshow(np.log(focal_int2), cmap='RdBu')
#     plt.title("Aberrated Coronagraphic PSF")
#     plt.colorbar()
    
#     plt.subplot(1,3,3)
#     plt.imshow(np.log(focal_int2), cmap='RdBu')
#     plt.title("Aberrated Coronagraphic PSF")
#     plt.colorbar()
    #cbar = plt.colorbar()
    #cbar.formatter.set_powerlimits((-14, 14))
    #cbar.ax.tick_params(labelsize = 15)
    #cbar.set_label("contrast",fontsize =15)
    #print(type(pupil_phase), type(focal_int))
    #plt.savefig('/Users/asahoo/Desktop/data_repos/harris_data/ball_del_06/plot_%d.png'%pp)
    #pf.writeto('/Users/asahoo/Desktop/data_repos/harris_data/ball_del_06/pupil_%d.fits'%pp,pupil_phase)
    #pf.writeto('/Users/asahoo/Desktop/data_repos/harris_data/ball_del_06/change_%d.fits'%pp, change) #save delta


In [ ]:
#1.6830731659218223e-14*600

In [ ]:
# plt.figure(figsize=(20,5))
# plt.title("Segment-based PASTIS constraints from PASTIS matrix and PASTIS modes")
# plt.plot((del_contrast))

## Plotting histograms 

In [ ]:
# n_harris = luvoir.harris_sm.num_actuators #int = 5*120 =600
# harris_mode =np.zeros(n_harris)
# luvoir.harris_sm.actuators = harris_mode

# unaber_e_field, ref, inter = luvoir.calc_psf(ref=True, return_intermediate='efield', norm_one_photon=True)



## Uncorrelated, random draw of mu_map_harris, average intensity of peak, photon noise contrast

In [ ]:
#mu_map_harris?
#np.savetxt('/Users/asahoo/Documents/ultra/report_11_01_21/mu_map.csv', mu_map_harris, delimiter=',')
#mu_map_harris = np.genfromtxt('/Users/asahoo/Desktop/data_repos/harris_data/mu_map.csv',delimiter=',')

In [ ]:
# all_contr_rand_seg1 = []
# for rep in range(10):
#     print('Segment realization {}/{}'.format(rep + 1, 50))
    
#     harris_modes = np.zeros(n_harris)
#     luvoir.harris_sm.actuators = harris_modes
    
#     input_wf = luvoir.wf_aper
#     harris_modes = np.random.normal(0,mu_map_harris*1e-9,n_harris)
#     luvoir.harris_sm.actuators = harris_modes/2
#     print(np.sqrt(np.mean(harris_modes**2)))
#     psf, ref, inter = luvoir.calc_psf(ref=True, return_intermediate='efield', norm_one_photon=True)
    
#     focal_int = np.zeros((115,115))
#     focal_int = np.array(np.reshape(psf.amplitude,(115,115)))
#     dh_intensity = ((np.square(psf.amplitude))/ norm) * luvoir.dh_mask
#     contrast_floor_aber = np.mean(dh_intensity[np.where(luvoir.dh_mask != 0)]) - contrast_floor
    
#     tmp0 = np.abs(np.mean(focal_int) - np.reshape(unaberrated_coro_psf,[115,115]))
#     #star_flux_5 = exoscene.star.bpgs_spectype_to_photonrate(spectype=sptype, Vmag=5, minlam=minlam.value, maxlam=maxlam.value)
#     #Nph_5 = star_flux_5.value*15**2*np.sum(luvoir.apodizer**2) / npup**2

#     #contrast2D = np.sqrt(tmp0)/norm/np.sqrt(Nph_5*0.04)
#     #contrast1D = np.reshape(contrast2D,[115**2])
#     #print(np.shape(contrast2D), np.shape(contrast1D))
#     print(np.mean(contrast1D[np.where(luvoir.dh_mask != 0)]))

    
    
#     print(dh_intensity, contrast_floor_aber)
#     all_contr_rand_seg1.append(contrast_floor_aber)
    
    
    #hcipy.imshow_field((inter['harris_seg_mirror']).phase, mask=luvoir_test.aperture, cmap='RdBu')
    #hcipy.imshow_field(np.log(np.square(aberrated_coro_psf_t.amplitude)), cmap='RdBu')

In [ ]:
# tmp_std = np.dot(matrix_pastis,np.diag(mu_map_harris)**2)
# var = 2*np.dot(tmp_std,tmp_std)
# std = np.sqrt(np.trace(var))
# c_mean_exp = np.mean(all_contr_rand_seg1)
# c_std_exp = np.std(all_contr_rand_seg1)
# print(std)

# fig = plt.figure(figsize=(10,10))
# plt.hist(np.log10(all_contr_rand_seg1),500)
# #plt.title('E2E raw contrast, {} realizations, target contrast 1e-11'.format(n_repeat), size=20)
# plt.xlabel('Mean contrast in DH', size=20)
# plt.xlim(-11.5,-10.5)
# plt.ylabel('Frequency', size=20)
# plt.axvline(c_target, c='r', ls='-.', lw='3')
# plt.axvline(c_target + std, c='b', ls=':', lw=6)
# plt.axvline(c_target - std, c='b', ls=':', lw=6)
# plt.axvline(c_mean_exp, c='orange', ls='-.', lw='3')
# plt.axvline(c_mean_exp + c_std_exp, c='c', ls=':', lw=6)
# plt.axvline(c_mean_exp - c_std_exp, c='c', ls=':', lw=6)
# plt.tick_params(axis='both', which='both', length=6, width=2, labelsize=25)
#plt.savefig('/Users/asahoo/Documents/ultra/temp_plots/hist_random1e-11.png')
#file_name = os.path.join(figures_path, f'{analysis_name}_MID_e2e_histogram.pdf')
#fig.savefig(file_name)

In [ ]:
# plt.plot(all_contr_rand_seg1)
# plt.ylabel("Delta Contrast")

## generate gif files 

In [ ]:
# import glob
# from PIL import Image

# # filepaths
# fp_in = "/Users/asahoo/Desktop/data_repos/harris_data/ball_del_01/*.png"
# fp_out = "/Users/asahoo/Desktop/data_repos/harris_data/image1.gif"

# # https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
# img, *imgs = [Image.open(f) for f in sorted(glob.glob(fp_in))]
# img.save(fp=fp_out, format='GIF', append_images=imgs,
#          save_all=True, duration=200, loop=0)


In [ ]:
# tmp0 = np.abs(np.mean(focal_int_array,0) - np.reshape(unaberrated_coro_psf,[115,115]))
# tmp = np.abs(focal_int_array[1] - np.reshape(unaberrated_coro_psf,[115,115]))
# plt.imshow(np.log10(tmp))
# plt.colorbar()

In [ ]:
# contrast2D = np.sqrt(tmp0)/norm/np.sqrt(Nph*3600)
# contrast1D = np.reshape(contrast2D,[115**2])

In [ ]:
# np.mean(contrast1D[np.where(luvoir.dh_mask != 0)])

In [ ]:
1e-11/5.60647816e-13

In [ ]:
# plt.imshow(np.log10(contrast2D))
# plt.colorbar()

In [ ]:
# plt.imshow(np.log10(focal_int_array[0]))
# plt.colorbar()

### Uncorrelated case with Sine wave random realizations

In [ ]:
# alpha_wfs = 0.13 #nm
# t_wfs = 10 #secs from the close loop curve

# n_trial = 1000
# focal_int_array_sin = np.zeros([n_trial,115,115])
# all_contr_correlated_sin = []

# for rep in range(n_trial):
#     print('Segment realization {}/{}'.format(rep + 1, n_trial))
    
#     harris_modes = mu_map_harris*1e-9*np.cos(2*np.pi/n_trial*rep)
#     luvoir.harris_sm.actuators = harris_modes/2    #18
    
#     psf, ref, inter = luvoir.calc_psf(ref=True, return_intermediate='efield', norm_one_photon=True)
    
#     focal_int_sin = np.zeros((115,115))
#     focal_int_sin = np.array(np.reshape(psf.intensity,(115,115)))
#     focal_int_array_sin[rep] = focal_int_sin
#     dh_intensity_sin = ((np.square(psf.amplitude))/ norm) * luvoir.dh_mask
#     contrast_floor_aber_sin = np.mean(dh_intensity_sin[np.where(luvoir.dh_mask != 0)]) - contrast_floor
    
#     print("sinewave frequency :",2*np.pi/n_trial*rep,"delta contrast", contrast_floor_aber_sin)
#     all_contr_correlated_sin.append(contrast_floor_aber_sin)

In [ ]:
# tmp_std = np.dot(matrix_pastis,np.diag(mu_map_harris)**2)
# var = 2*np.dot(tmp_std,tmp_std)
# std = np.sqrt(np.trace(var))
# s_mean_exp = np.mean(all_contr_correlated_sin)
# s_std_exp = np.std(all_contr_correlated_sin)
# #print(std)

# fig = plt.figure(figsize=(10,10))
# plt.hist(np.log10(all_contr_rand_seg1),500)
# #plt.title('E2E raw contrast, {} realizations, target contrast 1e-11'.format(n_repeat), size=20)
# plt.xlabel('Mean contrast in DH', size=20)
# plt.xlim(-11.5,-10.5)
# plt.ylabel('Frequency', size=20)
# plt.axvline(c_target, c='r', ls='-.', lw='3')
# plt.axvline(c_target + std, c='b', ls=':', lw=6)
# plt.axvline(c_target - std, c='b', ls=':', lw=6)
# plt.axvline(s_mean_exp, c='orange', ls='-.', lw='3')
# plt.axvline(s_mean_exp + s_std_exp, c='c', ls=':', lw=6)
# plt.axvline(s_mean_exp - s_std_exp, c='c', ls=':', lw=6)
# plt.tick_params(axis='both', which='both', length=6, width=2, labelsize=25)

In [ ]:
# plt.figure(figsize=(15,10))
# plt.plot(all_contr_correlated_sin)
# plt.xlabel("Spatial Frequency",fontsize=20)
# plt.ylabel("Dark hole mean contrast", fontsize=20)
# plt.tick_params(axis='both', which='both', length=6, width=2, labelsize=20)
# #plt.savefig('/Users/asahoo/Documents/ultra/report_11_01_21/sinwave_contrast.png')
# plt.grid()

In [ ]:
# tmp0_sin = np.abs(np.mean(focal_int_array_sin,0) - np.reshape(unaberrated_coro_psf,[115,115]))
# tmp_sin = np.abs(focal_int_array_sin[45] - np.reshape(unaberrated_coro_psf,[115,115]))
# plt.imshow(np.log10(tmp_sin))
# plt.colorbar()

In [ ]:
# star_flux_6 = exoscene.star.bpgs_spectype_to_photonrate(spectype=sptype, Vmag=6, minlam=minlam.value, maxlam=maxlam.value)
# Nph_6 = star_flux_6.value*15**2*np.sum(luvoir.apodizer**2) / npup**2

# contrast2D_sin = np.sqrt(tmp0_sin)/norm/np.sqrt(Nph_6*60)
# contrast1D_sin = np.reshape(contrast2D_sin,[115**2])

# print(np.shape(contrast2D_sin), np.shape(contrast1D_sin))
# print(np.mean(contrast1D_sin[np.where(luvoir.dh_mask != 0)]))


In [ ]:
#1e-11/4.2289512666549787e-10

In [ ]:
# plt.figure(figsize =(30,8))
# plt.subplot(1,3,1)
# plt.imshow(np.log10(tmp_sin))
# plt.colorbar()

# plt.subplot(1,3,2)
# plt.imshow(np.log10(contrast2D_sin))
# plt.colorbar()

# plt.subplot(1,3,3)
# plt.imshow(np.log10(focal_int_array_sin[0]))
# plt.colorbar()

### Code block to generate .gif files

In [ ]:
# for i in range(0, 120):
#     print(f'Working on "bulk" thermal mode, segment: {i+1}')
    
#     # Apply calibration aberration to used mode
#     harris_mode = np.zeros(n_harris)
#     harris_mode[i*5] = (nm_aber)/2 
#     luvoir.harris_sm.actuators  = harris_mode
    
#     plt.figure()
#     plt.text(9.6, 7.6, 'm', fontsize = 10)
#     hcipy.imshow_field(luvoir.harris_sm.surface, mask=luvoir.aperture, cmap='RdBu')
#     cbar = plt.colorbar()
#     plt.tight_layout()
#     #plt.savefig('/Users/asahoo/Documents/ultra/temp_plots/bulk_2_2_22/poke_%d.png'% i, dpi=165)
#     plt.savefig('/Users/asahoo/Documents/ultra/temp_plots/bulk_2_2_22/poke_{0:03}.png'.format(i), dpi=165)

In [ ]:
# from PIL import Image
# import glob

# imgs = glob.glob("/Users/asahoo/Documents/ultra/march_31st/gif_all/*.png")
# imgs.sort()
# frames = []

# for i in imgs:
#     new_frame = Image.open(i)
#     frames.append(new_frame)
    
# #Save into a GIF file that loops forever
# frames[0].save('/Users/asahoo/Documents/ultra/march_31st/mu_map_no_noise.gif', format='GIF',
#                append_images=frames[1:],
#                save_all=True,
#                duration=500, loop=0)